# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />   

### Tarea 2: Regresión y clasificación lineal

Aplica regresión lineal a los datos sintéticos que se encuentran divididos en los archivos de entre-
namiento x_entrenamiento.csv y y_entrenamiento.csv y los de validación x_validacion.csv
y y_validacion.csv. Estos datos fueron contaminados con ruido gaussiano con media igual a 0 y
desviación estándar igual a 0.05. Realiza lo siguiente:

- a. Grafica los datos de entrenamiento y de validación y comenta brevemente acerca de cómo
están distribuidos.

- b. Considera un modelo de la forma $f(x) = θ_0 + θ_1 · x_1 + θ_2 · x_2$ y realiza la regresión lineal.
Reporta los parámetros que encontraste usando el estimador de máxima verosimilitud y el
valor del error cuadrático medio para los datos de entrenamiento y de validación.

- c. Usa una expansión de base polinomial y entrena un modelo de regresión lineal con regulari-
zación por norma ℓ2. Reporta los parámetros obtenidos y el error cuadrático medio para los
datos de entrenamiento y de validación. bayesiana y obtén la distribución predictiva, repor-
tando la media y la varianza para el vector  ̃x= [1, 2]. Grafica la distribución a posteriori de
los parámetros con 5, 10, 30 y 60 datos. Adicionalmente, genera 20 muestras de la distribución
a posteriori de los parámetros con 5, 10, 30 y 60 datos y gráfica las curvas de los modelos para
estas muestras

In [59]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RepeatedKFold, cross_val_score

import plotly.express as px
import plotly.graph_objects as go

from tqdm import tqdm
import numpy as np

## Preprocesamiento de datos

In [32]:
# Cargar datos con pandas

x_entrenamiento = pd.read_csv('regl_data/x_entrenamiento.csv', header=None)
x_entrenamiento.columns = ['x1', 'x2']
y_entrenamiento = pd.read_csv('regl_data/y_entrenamiento.csv', header=None)
y_entrenamiento.columns = ['target']
x_validacion = pd.read_csv('regl_data/x_validacion.csv', header=None)
x_validacion.columns = ['x1', 'x2']
y_validacion = pd.read_csv('regl_data/y_validacion.csv', header=None)
y_validacion.columns = ['target']

X_train = pd.concat([x_entrenamiento, y_entrenamiento], axis=1)
X_val = pd.concat([x_validacion, y_validacion], axis=1)

# No hay datos nulos


In [33]:
X_train

,x1,x2,target
0,-1.913570,2.032600,0.619935
1,1.817693,2.787149,-0.640359
2,-1.287542,1.051852,1.874947
3,1.091132,2.287752,-0.365971
4,-0.952672,2.885976,0.623855
...,...,...,...
195,-0.871320,2.594933,1.042183
196,0.756463,2.392750,0.199176
197,-1.102479,1.766534,1.920034
198,0.493303,2.381998,0.456693


In [35]:
X_val

,x1,x2,target
0,0.522360,2.641996,0.489523
1,-1.556105,1.001378,1.051965
2,1.578210,0.965343,-1.055267
3,-1.796698,1.007622,0.492597
4,-1.778407,2.337267,0.660550
...,...,...,...
195,-1.088904,0.913317,2.236324
196,1.442120,1.809684,-1.036521
197,-1.612475,0.567502,0.565183
198,-0.133681,1.020684,2.761640


Al parecer no sabemos nada de los datos de entrada, así que vamos a ver qué hay en ellos mas que 

> "Estos datos fueron contaminados con ruido gaussiano con media igual a 0 y desviación estándar igual a 0.05"

### A.

In [100]:
# Exploración de datos

fig = px.scatter_3d(X_train, x='x1', y='x2', z='target')
fig.update_traces(marker=dict(size=5))
fig.update_layout(scene=dict(
    xaxis_title='x1',
    yaxis_title='x2',
    zaxis_title='target'
))
fig.show()

# Intersante... parece alguna función.

In [38]:
# Veamos la correlación entre las variables

correlation_matrix = X_train.corr()
fig = px.imshow(correlation_matrix, text_auto=True)
fig.update_layout(title='Correlation Matrix')
fig.show()

### B. Considera un modelo de la forma $f(x) = θ_0 + θ_1 · x_1 + θ_2 · x_2$ y realiza la regresión lineal.

Reporta los parámetros que encontraste usando el estimador de máxima verosimilitud y el
valor del error cuadrático medio para los datos de entrenamiento y de validación.

In [95]:
"""
Considera un modelo de la forma $f(x) = θ_0 + θ_1 · x_1 + θ_2 · x_2$ y realiza la regresión lineal.
Reporta los parámetros que encontraste usando el estimador de máxima verosimilitud y el
valor del error cuadrático medio para los datos de entrenamiento y de validación.
"""

def SquareTrainAndEval(X_train, X_test, y_train, y_test, model, degree=2):

    model = make_pipeline(PolynomialFeatures(degree), model)
    model.fit(X_train, y_train)

    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42) # Crea un objeto que realiza validación cruzada repetida.

    # Para Train
    y_train_pred = model.predict(X_train)
    mse_train = mean_squared_error(y_train, y_train_pred)
    r2_train = r2_score(y_train, y_train_pred) 
    cross_val_train = cross_val_score(model, X_train, y_train, cv=rkf) # Evalúa el modelo utilizando validación cruzada.
    cross_val_mean_train = cross_val_train.mean()
    cross_val_std_train = cross_val_train.std()

    # Para Test
    y_test_pred = model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_test_pred)
    r2_test = r2_score(y_test, y_test_pred)
    cross_val_test = cross_val_score(model, X_test, y_test, cv=rkf) # Evalúa el modelo utilizando validación cruzada.
    cross_val_mean_test = cross_val_test.mean()
    cross_val_std_test = cross_val_test.std()

    linear_model = model.named_steps[model.steps[1][0]]
    theta_0 = linear_model.intercept_
    coef = linear_model.coef_
    return {
        'theta_0': theta_0,
        'coef': coef,
        'mse_train': mse_train,
        'r2_train': r2_train,
        'cross_val_mean_train': cross_val_mean_train,
        'cross_val_std_train': cross_val_std_train,
        'mse_test': mse_test,
        'r2_test': r2_test,
        'cross_val_mean_test': cross_val_mean_test,
        'cross_val_std_test': cross_val_std_test
    }

# Definimos el modelo

model = LinearRegression()
# Entrenamos y evaluamos el modelo
results = SquareTrainAndEval(X_train[['x1', 'x2']], X_val[['x1', 'x2']], X_train['target'], X_val['target'], model)
# Imprimimos los resultados
results

{'theta_0': np.float64(2.3495820568863515),
 'coef': array([ 0.        , -0.97067567, -0.48838112, -0.61797171,  0.18448773,
         0.03135109]),
 'mse_train': 0.15247011551934306,
 'r2_train': 0.8603173118517873,
 'cross_val_mean_train': np.float64(0.8308660473907284),
 'cross_val_std_train': np.float64(0.06797604839328368),
 'mse_test': 0.2355025873126898,
 'r2_test': 0.8206199520751856,
 'cross_val_mean_test': np.float64(0.8308260175632927),
 'cross_val_std_test': np.float64(0.03980998410099421)}

In [96]:
import numpy as np
from scipy.optimize import minimize

class MLERegression:
    def __init__(self):
        self.intercept_ = None  # Término independiente (bias)
        self.coef_ = None       # Coeficientes de las características
        self.sigma2 = None      # Varianza de los errores

    def fit(self, X, y):
        # Aseguramos que X sea un array 2D
        X = np.asarray(X)
        y = np.asarray(y)
        self.n_samples, self.n_features = X.shape
        
        # Agregamos una columna de unos para el término independiente
        X_with_bias = np.hstack([np.ones((self.n_samples, 1)), X])
        
        # Inicializamos los parámetros (coeficientes + varianza)
        #initial_params = np.zeros(self.n_features + 2)  # Incluye intercept_, coef_ y sigma^2
        initial_params = np.random.randn(self.n_features + 2) * 0.01
        
        # Definimos la función de verosimilitud negativa
        def neg_log_likelihood(params):
            # Separar los coeficientes (incluyendo el término independiente) y la varianza
            theta = params[:-1]  # Coeficientes (incluye intercept_)
            sigma2 = params[-1]  # Varianza
            
            # Validar que la varianza sea positiva
            if sigma2 <= 0:
                return np.inf
            
            # Predicciones
            #y_pred = np.matmul(X_with_bias, theta)
            y_pred = X_with_bias @ theta

            
            # Residuos
            residuals = y - y_pred
            n = len(y)
            
            # Fórmula de la verosimilitud negativa para errores gaussianos
            return 0.5 * n * np.log(2 * np.pi * sigma2) + (0.5 / sigma2) * np.sum(residuals**2)
        
        # Optimizamos los parámetros utilizando el método de optimización de BFGS
        result = minimize(neg_log_likelihood, initial_params, method='BFGS')
        
        # Guardamos los parámetros optimizados
        self.intercept_ = result.x[0]  # Término independiente
        self.coef_ = result.x[1:-1]   # Coeficientes de las características
        self.sigma2 = result.x[-1]    # Varianza de los errores

    def predict(self, X):
        # Aseguramos que X sea un array 2D
        X = np.asarray(X)
        # Calculamos las predicciones
        #return np.matmul(X, self.coef_) + self.intercept_
        return X @ self.coef_ + self.intercept_

    def score(self, X, y):
        y_pred = self.predict(X)
        ss_total = np.sum((y - np.mean(y)) ** 2)
        ss_residual = np.sum((y - y_pred) ** 2)
        r2 = 1 - (ss_residual / ss_total)
        return r2

In [97]:
# Entrenamos y evaluamos el modelo

model = MLERegression()
mle_results = SquareTrainAndEval(X_train[['x1', 'x2']], X_val[['x1', 'x2']], X_train['target'], X_val['target'], model)

mle_results

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning:

invalid value encountered in subtract

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning:

invalid value encountered in subtract

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning:

invalid value encountered in subtract

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning:

invalid value encountered in subtract

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning:

invalid value encountered in subtract

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning:

invalid

{'theta_0': np.float64(1.1706925732985636),
 'coef': array([ 1.17888821, -0.97067553, -0.4883795 , -0.61797172,  0.18448766,
         0.03135065]),
 'mse_train': 0.15247011551938588,
 'r2_train': 0.8603173118517481,
 'cross_val_mean_train': np.float64(0.3000055855652721),
 'cross_val_std_train': np.float64(0.5754186724858649),
 'mse_test': 0.23550260664442121,
 'r2_test': 0.8206199373503923,
 'cross_val_mean_test': np.float64(0.10795228547776255),
 'cross_val_std_test': np.float64(0.7797415421080122)}

In [108]:
# Gráfica combinada de la superficie y los datos de entrenamiento

def plot_combined(theta_0, coef, X_train, degree=2):
    """
    Genera un gráfico 3D que combina la superficie ajustada y los datos de entrenamiento.

    Args:
        theta_0 (float): Término independiente.
        coef (list): Lista de coeficientes para las variables.
        X_train (DataFrame): Datos de entrenamiento con columnas 'x1', 'x2' y 'target'.
        degree (int): Grado del polinomio (por defecto 2).
    """
    x1_range = np.linspace(-3, 3, 5)
    x2_range = np.linspace(-3, 3, 5)

    X1, X2 = np.meshgrid(x1_range, x2_range)

    # Calcular Z dinámicamente según los coeficientes y el grado
    Z = theta_0
    coef_idx = 0
    for d in range(1, degree + 1):
        for i in range(d + 1):
            Z += coef[coef_idx] * (X1 ** (d - i)) * (X2 ** i)
            coef_idx += 1

    # Crear la figura combinada
    combined_fig = go.Figure()

    # Añadir la superficie de la función ajustada
    combined_fig.add_trace(go.Surface(z=Z, x=X1, y=X2, colorscale='Viridis', opacity=0.7, name='Superficie'))

    # Añadir los puntos de los datos de entrenamiento
    combined_fig.add_trace(go.Scatter3d(
        x=X_train['x1'],
        y=X_train['x2'],
        z=X_train['target'],
        mode='markers',
        marker=dict(size=5, color='blue', opacity=0.8),
        name='Datos de entrenamiento'
    ))

    # Configurar el diseño del gráfico
    combined_fig.update_layout(
        title='Superficie de la función ajustada y datos de entrenamiento',
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='target'
        )
    )

    # Mostrar la figura
    combined_fig.show()

# Llamada a la función con los parámetros
plot_combined(results['theta_0'], results['coef'], X_train, degree=2)
plot_combined(mle_results['theta_0'], mle_results['coef'], X_train, degree=2)

### c. Usa una expansión de base polinomial y entrena un modelo de regresión lineal con regularización por norma ℓ2. 

In [109]:
# c. Usa una expansión de base polinomial y entrena un modelo de regresión lineal con regularización por norma ℓ2. 

# Reporta los parámetros que encontraste usando el estimador de máxima verosimilitud y el valor del error cuadrático medio para los datos de entrenamiento y de validación.

from sklearn.linear_model import Ridge

model = Ridge(alpha=0.1)  # Regularización L2
# Entrenamos y evaluamos el modelo
ridge_results = SquareTrainAndEval(X_train[['x1', 'x2']], X_val[['x1', 'x2']], X_train['target'], X_val['target'], model, degree=20)
plot_combined(results['theta_0'], results['coef'], X_train)


/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning:

Singular matrix in solving dual problem. Using least-squares solution instead.

/opt/homebrew/Caskroom/miniforge/base/envs/datascience/lib/python3.13/site-packages/sklearn/linear_model/_ridge.py:254: UserWarning:

Singular m